In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import torch

In [ ]:
import json
import os
from glob import glob
# madalena_energy = pd.read_json("datasets/madalena/E-145/E145-august.json")

def extract_energy_json(file_path):
    save_path = "datasets/madalena_processed"
    os.makedirs(save_path, exist_ok=True)

    for house_path in os.listdir(file_path):
        dfs = []
        house_csv_path = house_path
        house_path = os.path.join(file_path, house_path)
        for file in glob(os.path.join(house_path, "*.json")):
            with open(file, "r", encoding="utf-8") as file_json:
                data = json.load(file_json)

            #because we have three phases (even though b,c seem to be 0)
            df_a = pd.DataFrame(data["energy_a"]).rename(columns={"value": "energy_a"})
            df_b = pd.DataFrame(data["energy_b"]).rename(columns={"value": "energy_b"})
            df_c = pd.DataFrame(data["energy_c"]).rename(columns={"value": "energy_c"})

            df_b["timestamp"] = pd.to_datetime(df_b["time"])
            df_c["timestamp"] = pd.to_datetime(df_c["time"])
            df_a["timestamp"] = pd.to_datetime(df_a["time"])

            energy_df = df_a.merge(df_b, on="timestamp", how="inner").merge(df_c, on="timestamp", how="inner")
            #remove duplicates
            energy_df = energy_df.loc[:, ~energy_df.columns.duplicated()]


            energy_df["total_energy"] = energy_df["energy_a"] + energy_df["energy_b"] + energy_df["energy_c"]
            energy_df = energy_df[["timestamp", "total_energy"]].set_index("timestamp")
            dfs.append(energy_df)
        df = pd.concat(dfs)
        df.to_csv(f"{save_path}/{house_csv_path}.csv")

extract_energy_json("datasets/madalena")

In [16]:
import json
import os
from glob import glob
#get comfort metrics and concat them with energy data per house
#energy and comfort might not start and end at exaclty the same time, so we also need to handlle that
def merge_madalena():
    madalena_comfort_df = pd.read_csv("datasets/madalena/madalena_comfort.csv")
    madalena_comfort_df["timestamp"] = pd.to_datetime(madalena_comfort_df["date"])
    madalena_comfort_df.drop(columns=["date"], inplace=True)
    madalena_comfort_df.set_index("timestamp", inplace=True)

    save_dir = "datasets/madalena_merged"
    os.makedirs(save_dir, exist_ok=True)
    energy_path = "datasets/madalena_processed"

    for file in os.listdir(energy_path):
        madalena_energy_df = pd.read_csv(f"{energy_path}/{file}")
        madalena_energy_df["timestamp"] = pd.to_datetime(madalena_energy_df["timestamp"])
        madalena_energy_df.set_index("timestamp", inplace=True)
        if madalena_energy_df.index.tz is not None:
            madalena_energy_df.index = madalena_energy_df.index.tz_convert(None)
        madalena_energy_df = madalena_energy_df.sort_values(by="timestamp")
        house_name = file.split(".")[0].split("_")[1]
        house_name = f"E{house_name}"
        madalena_comfort_house = madalena_comfort_df[madalena_comfort_df["Room"] == house_name].copy()
        madalena_comfort_house.drop(columns=["Room"], inplace=True)
        start_time = max(madalena_energy_df.index.min(), madalena_comfort_house.index.min())
        end_time = min(madalena_energy_df.index.max(), madalena_comfort_house.index.max())

        madalena_energy_df = madalena_energy_df.loc[start_time:end_time]
        madalena_comfort_house = madalena_comfort_house.loc[start_time:end_time]
        print(madalena_energy_df.head(20), madalena_comfort_df.head(20))

        madalena_merged_df = madalena_energy_df.merge(madalena_comfort_house, left_index=True, right_index=True, how="inner")
        print(madalena_merged_df.head(20))
        # madalena_merged_df.to_csv(f"{save_dir}/{house_name}.csv")

merge_madalena()

                     total_energy
timestamp                        
2024-07-01 00:00:00         71.77
2024-07-01 01:00:00         94.17
2024-07-01 02:00:00         83.25
2024-07-01 03:00:00         84.62
2024-07-01 04:00:00         92.07
2024-07-01 05:00:00        209.27
2024-07-01 06:00:00        515.45
2024-07-01 07:00:00         66.00
2024-07-01 08:00:00        106.00
2024-07-01 09:00:00        246.63
2024-07-01 10:00:00        544.17
2024-07-01 11:00:00        137.53
2024-07-01 12:00:00         67.62
2024-07-01 13:00:00        120.28
2024-07-01 14:00:00        113.67
2024-07-01 15:00:00        141.30
2024-07-01 16:00:00        245.20
2024-07-01 17:00:00        208.23
2024-07-01 18:00:00        208.18
2024-07-01 19:00:00        206.80                      Room  CO2[ppm]  PM4[ug/m3]  Lighting[lux]  T_in[°C]  \
timestamp                                                                  
2024-07-01 00:00:00  E145    624.23       65.43           0.00     28.46   
2024-07-01 01:00:00  E14

In [9]:
df_145 = pd.read_csv("datasets/madalena_merged/E145.csv")
df_146 = pd.read_csv("datasets/madalena_merged/E146.csv")
df_147 = pd.read_csv("datasets/madalena_merged/E147.csv")
df_148 = pd.read_csv("datasets/madalena_merged/E148.csv")

dfs = [df_145, df_146, df_147, df_148]
dfs_names = ["145", "146", "147", "148"]
#check nulls
# print(df_145.isnull().sum())
# print(df_146.isnull().sum())
# print(df_147.isnull().sum())
# print(df_148.isnull().sum())

#we have none so no need to remove any rows
for id, df in enumerate(dfs):
    df["house_id"] = dfs_names[id]

In [11]:
sample_df = pd.concat(dfs)
sample_df.sort_values(by='timestamp').set_index('timestamp')
sample_df.to_csv("datasets/madalena_sample.csv")
